# Convergence and Optimality Analysis of Low-Dimensional Generative Adversarial Networks using Error Function Integrals

## Requirements

Use `requirements.txt` file preferably in virtual environment.

In order to do that in the current folder, for Ubuntu:
```
$ sudo apt install virtualenv
$ virtualenv venv
$ source venv/bin/activate
(venv)$ pip install -r requirements.txt
(venv)$ python -m ipykernel install --user --name=venv
```

Launch from command prompt: 
```bash
(venv)$ jupyter notebook
```

or open it within VSCode with [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) or any other favorite method. Please make sure that the Jupyter kernel points to your environment (select it from the kernel menu).

## Imports

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys

from scipy.io import savemat 
from tqdm import tqdm

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
sys.path.append('src')

In [13]:
from utils import load_analytical, show_comparative, create_3d_plot
from sgd import SGD_gh
from gan import Jgh, sample_Jgh

## Common notes
Objective function (from Goodfellow's paper):
$$
V[G,D] = \mathbb{E}_x \log D(x) + \mathbb{E}_z \log(1 - D(G(z))
$$

The goal is to optimize
$$
\min_G \max_D V[G, D]
$$

The assumption of Goodfellow's paper is that we can find an algorithm which finds globally optimal $D^*, G^*$. Let's see if we can optimize for some simple case, introducing some new levels of complexity if we need them. If the approach of Gf is formulated for a very common case, it should work for a basic one.

A least squares GAN cost function is:
$$
V[G, D] = E_x D^2(x) + E_z [1 - D(G(z))]^2
$$

# Analysis of some case of `x` and `z` distribution


Here we assume
$$
x \sim \text{Exp}(c) \\
z \sim \text{Rayleigh}(1) \\
D(x) = {1 + \Phi(ax + b) \over 2} \\
G(z) = g z^2 + h
$$

Where
$$
\text{Exp}(c) = c e^{-c x} \\
\text{Rayleigh}(1) = 2x \exp(-x^2)
$$

Augmented cost function
$$
V[D,G] = \mathbb{E}_x D^2(x) + \mathbb{E}_z [1 - D(G(z))]^2 = J_1[D] + J_2[D,G]
$$

Therefore
$$
J_1[D] = \frac{1}{4}\int_0^{\infty} (1+\Phi(ax + b))^2 \text{Exp}(x|c) dx \\
J_2[D, G] = \\
\frac{1}{4g}\int_0^{\infty} [1 - \Phi(a\zeta + \eta)]^2 \text{Exp}(\zeta|1/g) dz
$$
where $\eta=ah+b$ and $\zeta=gz^2$


## Cost function computed using sampling
Easiest way (the one, which Goodfellow also follows):
$$
V[G,D] = {1 \over m} \sum_{i = 1}^m
D^2(x_i) + [1 - D(G(z_i))]^2
$$

## SGD implementation
Here we implement vanilla SGD for the example above.

${\partial J \over \partial g}$, ${\partial J \over \partial h}$ are defined in `sgd.py` module.

Single SGD step:
$$
g \leftarrow g + \epsilon_g {\partial J \over \partial g} \\
h \leftarrow h - \epsilon_h {\partial J \over \partial h}
$$

SGD procedure:

In [14]:
def run_SGD(config_sgd, config_jbh, jbh_surface, gg, hg, x, z, plot_result=True):
  a, b, c, _, _ = [v[1] for v in config_jbh.items()]
  g_initial, h_initial, eps_b, eps_h, n_steps = [v[1] 
                                                 for v in config_sgd.items()]
  gd = SGD_gh(x, z, a, b, c, g_initial, h_initial, eps_b, eps_h, n_steps)
  path = [val for val in gd]
  jgh_path = [Jgh(x, z, a, b, c, val[0], val[1]) for val in path]
  g_path = [val[0] for val in path]
  h_path = [val[1] for val in path]
  
  if plot_result:
    _, ax = create_3d_plot()
    ax.plot_wireframe(gg, hg, jbh_surface)
    ax.plot(g_path, h_path, jgh_path, 'r--')
    ax.text(g_path[0], h_path[0], jgh_path[0],
            r'$\leftarrow b_0, h_0 = (%1.1f, %1.1f)$'
            % (g_path[0], h_path[0]), fontsize=16)
    n_path = len(path) - 1
    ax.text(g_path[n_path], h_path[n_path], jgh_path[n_path],
            r'$\leftarrow \hat b, \hat h = (%1.1f, %1.1f)$' % 
            (g_path[n_path], h_path[n_path]), fontsize=16)
    plt.xlabel('g')
    plt.ylabel('h')
    plt.title(
        r'SGD w.r.t. $(g,h)$ $a = %1.1f, b = %1.1f, c = %1.1f, '
        '\epsilon_g = %1.1f, \epsilon_h = %1.1f$, %d steps' 
        % (a, b, c, eps_b, eps_h, n_steps))
    plt.show()
  return g_path, h_path

In [15]:
def run_SGD_batch(jgh_config, sgd_config, n_runs=100, n_samples=1000, plot_result=False):
  g_paths = []
  h_paths = []
  for _ in tqdm(range(n_runs)):
    jbh, gg, hg, x, z = sample_Jgh(jgh_config, n_samples=n_samples)
    g_hat, h_hat = run_SGD(sgd_config, jgh_config, 
                           jbh, gg, hg, x, z,
                           plot_result=plot_result)
    g_paths += [g_hat]
    h_paths += [h_hat]
  return g_paths, h_paths

### Test case implementation
Configuration:

In [16]:
jgh_config = {
    "a": 2.2,
    "b": 1.0,
    "c": 0.5,
    "g": np.linspace(0.1, 3.0, 61), 
    "h": np.linspace(-2.1, 3.1, 41),
}
sgd_config = {
    "g_initial": 0.2,
    "h_initial": -2.0,
    "eps_g": 0.4,
    "eps_h": 0.4,
    "n_steps": 250
}

In [17]:
show_intermediate_steps = False

MC SGD experiment:

In [18]:
def test_case_gh(n_runs):
  g_paths, h_paths = run_SGD_batch(jgh_config, sgd_config,
                                   n_runs=n_runs, plot_result=show_intermediate_steps)
  show_comparative(g_paths, h_paths,
                   thetas=load_analytical('content/Thetas_A.mat', 'Thetas_A'))
  savemat('content/test_case_gh.mat', {'g_paths': g_paths, 'h_paths': h_paths})

test_case_gh(100)

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:11<18:14, 11.05s/it]